# Post-wildfire vegetation regrowth analysis
Author: Stepan Bryleev

### Normalized Difference Moisture Index (NDMI)

In this Jupyter Notebook there is a JavaScript code for calculating and displaying Normalized Difference Moisture Index (NDMI) values for the study area in GEE using NEON reflectance data.

* NDMI is used to determine vegetation water content. It is calculated as a ratio between the NIR and SWIR values in traditional fashion. \
    NEON spectrometer band B172 (1240 nm) is used for SWIR2.
* $NDMI=\frac{(NIR-SWIR2)}{(NIR+SWIR2)}$
* The difference normalized burn ratio (dNDMI) is the difference between the pre-fire NDMI and the post-fire NDMI.

```js
// ======================================================= //
// This part of the project is for creating GRSM NDMI maps //
// ======================================================= //

// Specify center location and for GRSM
var SiteCenterPoint = ee.Geometry.Point([-83.5, 35.7]);

// Center the map on our area of interest and set zoom level
Map.setCenter(-83.5, 35.7, 12);

var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perim variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);
print('polygon intersection',polygonIntersection);

// Create study area.
var studyArea = study_polygon.intersection(polygonIntersection);

// Print polygon area in square kilometers.
print('Study area square: ', studyArea.area().divide(1000 * 1000));

// Print polygon perimeter length in kilometers.
print('Study area perimeter: ', studyArea.perimeter().divide(1000));


// ===== Create a cloud masking function ===== //

// Define a function
function neon_mask(image) {
  // Create a single band Weather Quality QA layer & Mask out cloudy pixels from the image
  var weather_qa = image.select(['Weather_Quality_Indicator']);
  var clear_qa = weather_qa.gt(0);
  var nis_image_clear = image.updateMask(clear_qa);
  return nis_image_clear;
}


// === Create a function to calculate and display NDMI image with a date === //

// NDMI = (NIR – SWIR1) / (NIR + SWIR1)

// NIR = NIS Band B097
// SWIR2 = NIS Band B172 (1240 nm)

// Define a function and bands of interest
function normdiff_ndmi(image, date) {
  // NIR = NIS Band B097, SWIR2 = NIS Band B172
  var nis_normdiff = image.normalizedDifference(["B097", "B246"]);
  var viz_params = {min:-1, max:1, palette:['brown', 'pink', 'lightgreen', 'darkgreen']};
  Map.addLayer(nis_normdiff, viz_params, "GRSM NDMI " + date, 0);
  
  return nis_normdiff;
}


// === Create a function to calculate dNDMI image multipied by 1000 === //

// Define a function
function calc_dNDMI(pre_fire_im, post_fire_im) {
  // Subtract images and multiply by 1000
  var dNDMI_unscaled = pre_fire_im.subtract(post_fire_im);
  var dNDMI_image = dNDMI_unscaled.multiply(1000);

  return dNDMI_image;
}


// === Create a function to plot histogram charts === //

// Define a function
function plot_hist(diff_image, studyArea, scale, title) {
  var unscaled_im = diff_image.divide(1000);
  var diff_hist = ui.Chart.image.histogram({image: unscaled_im, region: studyArea, scale: scale})
      .setOptions({title: title,
            hAxis: {title: 'dNDMI Value',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});
            
  print(diff_hist);
}

// DEFINE DATES OF INTEREST:

var date_2016 = '2016-06-08';
var date_2017 = '2017-10-05'; 
var date_2021 = '2021-06-18';


//  DATA COLLECTION: 'projects/neon-prod-earthengine/assets/DP3-30006-001'

// ======= Read 2016 data ======= //

// Read in the 2016 SDR Image for GRSM
var NISimage_2016 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2016_GRSM_2_SDR')
                      .clip(studyArea);
// Apply masking function to 2016 image
var nis_image_2016_masked = neon_mask(NISimage_2016);

print(NISimage_2016);
// ======= Read 2017 data ======= //

// Read in the 2017 SDR Image for GRSM
var NISimage_2017 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2017_GRSM_3_SDR')
                      .clip(studyArea);
// Apply masking function to 2017 image
var nis_image_2017_masked = neon_mask(NISimage_2017);


// ======= Read 2021 data ======= //

// Read in the 2021 SDR Image for GRSM
var NISimage_2021 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2021_GRSM_5_SDR')
                      .clip(studyArea);
// Apply masking function to 2021 image
var nis_image_2021_masked = neon_mask(NISimage_2021);


// ======= Calculate and plot NDMI for each date of interest ======= //

var pre_fire_ndmi_2016 = normdiff_ndmi(nis_image_2016_masked, date_2016);
var post_fire_ndmi_2017 = normdiff_ndmi(nis_image_2017_masked, date_2017);
var post_fire_ndmi_2021 = normdiff_ndmi(nis_image_2021_masked, date_2021);


// ======= Calculate dNDMI images of difference ======= //

var dNDMI_2017_2016 = calc_dNDMI(pre_fire_ndmi_2016, post_fire_ndmi_2017);
var dNDMI_2021_2016 = calc_dNDMI(pre_fire_ndmi_2016, post_fire_ndmi_2021);


// ======= Create dNDMI Classification and add dNDMI images to the map ======= //

// SLD = Styled Layer Descriptor

// Define an SLD style of discrete intervals to apply to the image.
var sld_intervals =
  '<RasterSymbolizer>' +
    '<ColorMap type="intervals" extended="false" >' + 
      '<ColorMapEntry color="#008080" quantity="0" label="0"/>' + // Teal
      '<ColorMapEntry color="#3CB371" quantity="100" label="100" />' + // MediumSeaGreen
      '<ColorMapEntry color="#FFEFD5" quantity="200" label="200" />' + // PapayaWhip
      '<ColorMapEntry color="#FFC0CB" quantity="300" label="300" />' + // Pink
      '<ColorMapEntry color="#FF1493" quantity="400" label="400" />' + // DeepPink
      '<ColorMapEntry color="#FF0000" quantity="500" label="500" />' + // Red
      '<ColorMapEntry color="#FF00FF" quantity="600" label="600" />' + // Magenta
      '<ColorMapEntry color="#8B008B" quantity="1000" label="1000" />' + // DarkMagenta

    '</ColorMap>' +
  '</RasterSymbolizer>';

// Add dNDMI images to the map using both the color ramp and interval schemes
Map.addLayer(dNDMI_2017_2016.sldStyle(sld_intervals), {}, '2017/2016 dNDMI classified');
Map.addLayer(dNDMI_2021_2016.sldStyle(sld_intervals), {}, '2021/2016 dNDMI classified');


// =======   ADD A dNDWI LEGEND  ======== //

// Create legend panel and set its position
var legend = ui.Panel({
  style: {
    position: 'bottom-left',
    padding: '8px 20px'
  }});
 
// Create legend title
var legendTitle = ui.Label({
  value: 'dNDMI Values',
  style: {fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 6px 0',
    padding: '0'
    }});
 
// Add the title to the panel
legend.add(legendTitle);
 
// Creates and styles 1 row of the legend.
var makeRow = function(color, name) {
 
      // Create the label that is actually the colored box.
      var colorBox = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Use padding to give the box height and width.
          padding: '8px',
          margin: '0 0 4px 0'
        }});
 
      // Create the label filled with the description text.
      var description = ui.Label({
        value: name,
        style: {margin: '0 0 4px 6px'}
      });
 
      // Return the panel
      return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
      })};
 
//  Palette with the colors
var palette =['008080', '3CB371', 'FFEFD5', 'FFC0CB', 'FF1493', 'FF0000', 'FF00FF', '8B008B'];
 
// Names of the legend
var names = ['Positive change', '0 ÷ - 0.1', '- 0.1 ÷ - 0.2', '- 0.2 ÷ - 0.3',
              '- 0.3 ÷ - 0.4', '- 0.4 ÷ - 0.5', '- 0.5 ÷ - 0.6', 'Diff > - 0.6'];

// Add color and and names to a legend panel
for (var i = 0; i < 8; i++) {
  legend.add(makeRow(palette[i], names[i]));
  }  
 
// Add legend to map (alternatively you can also print the legend to the console)
Map.add(legend);


// ======= Display Fire Boundary ======= //

// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);
                                  

// ======= Plot histogram charts ======= //

// Plot histogram charts for each dNDVI image in the console
plot_hist(dNDMI_2017_2016, studyArea, 50, 'dNDMI 2016/2017');
plot_hist(dNDMI_2021_2016, studyArea, 50, 'dNDMI 2016/2021');

```